# Reproducing the baseline model results from the paper <a class="tocSkip">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Download-the-ACME-dataset-used" data-toc-modified-id="Download-the-ACME-dataset-used-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Download the ACME dataset used</a></span></li><li><span><a href="#Fit-all-the-models" data-toc-modified-id="Fit-all-the-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fit all the models</a></span></li><li><span><a href="#Evaluate-the-models-and-produce-results-tables-and-figures" data-toc-modified-id="Evaluate-the-models-and-produce-results-tables-and-figures-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Evaluate the models and produce results tables and figures</a></span></li></ul></div>

tl;dr: run the grid searches (all commands in `experiments.txt`), then run the evaluation with `baselines/eval_task.py`.

# Download the ACME dataset used

1. Go to https://github.com/JamesOwers/acme, select the version tag required, and extract the zip into the root directory of the repo
1. you should now have the directory structure:
```
midi_degradation_toolkit
    mdtk
    baselines
    ...
    acme
        altered
        clean
        ...
        valid_pr_corpus.csv
```

# Fit all the models

1. To fit all the models, run all of the commands contained within the file
[`experiment.txt`](./experiment.txt). These are all python commands which
expect you to have installed the package.
1. we recommend running from the base directory and saving the outputs to
a folder there. For example, to run the first 5 experiments:
```
repo_dir=..
cd $repo_dir
head -5 baselines/experiment.txt | bash
```
1. You should now have a folder `$repo_dir/output` which contains
subdirectories with the name of each task
1. in each subdirectory, there are model checkpoint files and the
training log
1. Tip: if you want to save the results somewhere else, edit
`experiment.txt` using `sed`:
```
sed -i 's_output_some/other/place_g' baselines/experiment.txt
```
1. Tip: we used distributed computing with `slurm` (approx 20 experiments at
a time) to run our experiments in parallel. Each node had a GPU. In this
setting, it took about 1 day to run all the experiments in
`experiment.txt`

# Evaluate the models and produce results tables and figures

You can produce plots and tables from experiments that have been run by running
[`./baselines/get_results.py`]:
1. Assuming you now have an directory structure as outlined above
and you ran all the experiments in `experiment.txt`, you can run
the following command to produce all the plots and results tables
```bash
python ./baselines/get_results.py \
    --output_dir output \
    --save_plots output \
    --in_dir acme \
    --task_names task1 task1weighted task2 task3 task3weighted task4 \
    --setting_names "['lr','wd','hid']" "['lr','wd','hid']" "['lr','wd','hid']" "['lr','wd','hid']" "['lr','wd','hid']" "['lr','wd','hid','lay']" \
    --formats command command command pianoroll pianoroll pianoroll \
    --seq_len 1000 1000 1000 250 250 250 \
    --metrics rev_f rev_f avg_acc f f helpfulness \
    --task_desc ErrorDetection ErrorDetection ErrorClassification ErrorLocation ErrorLocation ErrorCorrection \
    --splits train valid test
```
1. This will load the models and perform evaluation, so it is again
recommended that you execute the above command with GPU availability
1. Observe the results!
    * The output directory now contains additional plots and tables
    * For example:
    ```
    output
        task1__all_loss_summary.pdf  <- all of the loss curves in one plot
        task1__best_model_loss.pdf   <- the loss curves for the best model
        task1__min_loss_summary.pdf  <- of all models, a summary of the minimum
                                        validation losses they attained over
                                        each setting
        task1_table.tex              <- full results for train vaild test split
        ...
        summary_table.tex            <- superset of table in the paper
    ```